In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
import sys
#sys.path.append('/usr/local/google/home/fprost/Documents/Projects/jigsaw/conversationai-models/experiments')

In [ ]:
"""Experiments with Toxicity Dataset"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Import common flags and run code. Must be imported first.
#from tf_trainer.common import model_trainer

from tf_trainer.common import tfrecord_input
from tf_trainer.common import serving_input
from tf_trainer.common import text_preprocessor
from tf_trainer.common import types
from tf_trainer.keras_gru_attention import model as keras_gru_attention

import nltk
import tensorflow as tf

from typing import Dict

In [ ]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string("embeddings_path",
                           "gs://kaggle-model-experiments/resources/glove.6B/glove.6B.300d.txt",
                           "Path to the embeddings file.")
tf.app.flags.DEFINE_boolean("is_binary_embedding", False,
                           "Whether embeddings are binaries.")
tf.app.flags.DEFINE_string("text_feature_name", "comment_text",
                           "Feature name of the text feature.")
tf.app.flags.DEFINE_string("key_name", "comment_key",
                           "Name of the key feature for serving examples.")
tf.app.flags.DEFINE_boolean("preprocess_in_tf", False,
                           "Run preprocessing with TensorFlow operations,"
                           "required for serving.")
tf.app.flags.DEFINE_integer("batch_size", 5,
                            "The batch size to use during training.")
tf.app.flags.DEFINE_integer("train_steps", 20000,
                            "The number of steps to train for.")
tf.app.flags.DEFINE_integer("eval_period", 100,
                            "The number of steps per eval period.")
tf.app.flags.DEFINE_integer("eval_steps", 20,
                            "The number of steps to eval for.")


# TODO: Missing fields are not handled properly yet.
LABELS = {
    "frac_neg": tf.float32,
    #"frac_very_neg": tf.float32
}  # type: Dict[str, tf.DType]

In [ ]:
def main(argv):
  del argv  # unused

In [ ]:
try:
    tf.logging.set_verbosity(tf.logging.INFO)
    tf.app.run(main)
except:
    pass

# Glove embeddings

In [ ]:
embeddings_path = 'gs://kaggle-model-experiments/resources/glove.6B/glove.6B.300d.txt'
is_binary_embedding = False
text_feature_name = FLAGS.text_feature_name
key_name = FLAGS.key_name

preprocessor = text_preprocessor.TextPreprocessor(embeddings_path, is_binary_embedding)

## Preprocess in python

In [ ]:
preprocess_in_tf = False

if preprocess_in_tf:
  tokenize_op_init = lambda: preprocessor.tokenize_tensor_op_tf_func()
else:
  nltk.download("punkt")
  tokenize_op_init = lambda: preprocessor.tokenize_tensor_op_py_func(nltk.word_tokenize)
dataset = tfrecord_input.TFRecordInput(
  train_path="gs://kaggle-model-experiments/resources/toxicity_q42017_train.tfrecord",
  validate_path="gs://kaggle-model-experiments/resources/toxicity_q42017_train.tfrecord",
  text_feature=text_feature_name,
  labels=LABELS,
  feature_preprocessor_init=tokenize_op_init,
  batch_size=FLAGS.batch_size)

input_fn = dataset.train_input_fn

In [ ]:
import numpy as np

prop_list = []
el = input_fn()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(tf.tables_initializer())
    for i in range(500):
      d = sess.run(el)
      d = d[0]
      for i in range(d['comment_text'].shape[0]):
            prop = np.mean([word==400001 for word in d['comment_text'][i]])
      prop_list.append(prop)
      #print (d)
print ('FINAL PROPORTION')
print (np.mean(prop_list))

## Preprocess in tf

In [ ]:
preprocess_in_tf = True

if preprocess_in_tf:
  tokenize_op_init = lambda: preprocessor.tokenize_tensor_op_tf_func()
else:
  nltk.download("punkt")
  tokenize_op_init = lambda: preprocessor.tokenize_tensor_op_py_func(nltk.word_tokenize)
dataset = tfrecord_input.TFRecordInput(
  train_path="gs://kaggle-model-experiments/resources/toxicity_q42017_train.tfrecord",
  validate_path="gs://kaggle-model-experiments/resources/toxicity_q42017_train.tfrecord",
  text_feature=text_feature_name,
  labels=LABELS,
  feature_preprocessor_init=tokenize_op_init,
  batch_size=FLAGS.batch_size)

input_fn = dataset.train_input_fn

In [ ]:
import numpy as np

prop_list = []
el = input_fn()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(tf.tables_initializer())
    for i in range(500):
      d = sess.run(el)
      d = d[0]
      for i in range(d['comment_text'].shape[0]):
            prop = np.mean([word==400001 for word in d['comment_text'][i]])
      prop_list.append(prop)
      #print (d)
print ('FINAL PROPORTION')
print (np.mean(prop_list))

# Google-news embeddings

In [ ]:
embeddings_path = 'gs://kaggle-model-experiments/resources/google-news/GoogleNews-vectors-negative300.txt'
is_binary_embedding = True
text_feature_name = FLAGS.text_feature_name
key_name = FLAGS.key_name

preprocessor_google = text_preprocessor.TextPreprocessor(embeddings_path, is_binary_embedding)

## Preprocess in python

In [ ]:
preprocess_in_tf = False

if preprocess_in_tf:
  tokenize_op_init = lambda: preprocessor_google.tokenize_tensor_op_tf_func()
else:
  nltk.download("punkt")
  tokenize_op_init = lambda: preprocessor_google.tokenize_tensor_op_py_func(nltk.word_tokenize)
dataset = tfrecord_input.TFRecordInput(
  train_path="gs://kaggle-model-experiments/resources/toxicity_q42017_train.tfrecord",
  validate_path="gs://kaggle-model-experiments/resources/toxicity_q42017_train.tfrecord",
  text_feature=text_feature_name,
  labels=LABELS,
  feature_preprocessor_init=tokenize_op_init,
  batch_size=FLAGS.batch_size)

input_fn = dataset.train_input_fn

In [ ]:
import numpy as np

prop_list = []
el = input_fn()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(tf.tables_initializer())
    for i in range(500):
      d = sess.run(el)
      d = d[0]
      for i in range(d['comment_text'].shape[0]):
            prop = np.mean([word==3000001 for word in d['comment_text'][i]])
      prop_list.append(prop)
      #print (d)
print ('FINAL PROPORTION')
print (np.mean(prop_list))

## Preprocess in tf

In [ ]:
preprocess_in_tf = True

if preprocess_in_tf:
  tokenize_op_init = lambda: preprocessor_google.tokenize_tensor_op_tf_func()
else:
  nltk.download("punkt")
  tokenize_op_init = lambda: preprocessor_google.tokenize_tensor_op_py_func(nltk.word_tokenize)
dataset = tfrecord_input.TFRecordInput(
  train_path="gs://kaggle-model-experiments/resources/toxicity_q42017_train.tfrecord",
  validate_path="gs://kaggle-model-experiments/resources/toxicity_q42017_train.tfrecord",
  text_feature=text_feature_name,
  labels=LABELS,
  feature_preprocessor_init=tokenize_op_init,
  batch_size=FLAGS.batch_size)

input_fn = dataset.train_input_fn

In [ ]:
import numpy as np

prop_list = []
unknown_words = []
el = input_fn()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(tf.tables_initializer())
    for i in range(50):
      d = sess.run(el)
      d = d[0]
      for i in range(d['comment_text'].shape[0]):
            prop = np.mean([word==3000001 for word in d['comment_text'][i]])
            for j, word in enumerate(d['comment_text'][i]):
                if word == 3000001:
                    unknown_words.append(d['words'][i][j])
      prop_list.append(prop)
      #print (d)
print ('FINAL PROPORTION')
print (np.mean(prop_list))

In [ ]:
unknown_words

In [ ]:
text = tf.placeholder(shape=[1], dtype=tf.string)
text = tf.regex_replace(text, '\n', ' ')
text = tf.regex_replace(text, '\.', ' ')
text = tf.regex_replace(
  text,
  '[!"#$%&()*+,-/:;<=>?@[\]^_`{|}~]',
  ' ')
words = tf.string_split(text)
words = tf.sparse_tensor_to_dense(words, default_value='')

In [ ]:
sess = tf.Session()

sess.run(words, feed_dict={text: ['This is a test']})